## Классы
Класс - это шаблон для создания объектов

Класс состоит из **членов**  
Члены это:
1. Методы - функции класса
2. Поля - переменные класса
Создание класса:
```python
class <название класса>:
    <поля и методы класса>
```

In [1]:
class A:
    pass

In [2]:
a = A()

In [6]:
# type - возвращает тип
# isinstance(x, y) возвращает True если x принадлежит классу y
type(1.0), isinstance('a', str)

(float, True)

In [22]:
class Student:
    # __init__ - метод инициализации объекта 
    def __init__(self, name, age = 18):
        self.name = name
        self.age = age
        
        self.group = 'itam_python_courses'
        
    def return_age(self):
        return self.age

In [57]:
artem = Student('artem', 25)
ivan = Student('ivan_xxx', 0)

In [35]:
class Student:
    # данное поле - статическое
    # статическое поле - доступное через имя класса
    _count_students = 0
    
    
    def __init__(self, name, age = 18):
        self.name = name
        self.age = age
        Student._count_students += 1

    # @staticmethod - делает метод статическим 
    @staticmethod
    def amount():
        return Student._count_students

In [53]:
a, b, c = Student('peter'), Student('seva'), Student('michael')

In [60]:
Student.amount()

14

In [62]:
class Student:
    _count_students = 0
    
    
    def __init__(self, name, age = 18):
        self.name = name
        self.age = age
        Student._count_students+=1

    # Через self можно обращаться к полям объекта
    def return_data(self):
        return self.name + ", " + str(self.age)

In [63]:
a = Student('Ольга')

In [64]:
a.return_data()

'Ольга, 18'

In [66]:
class Student:
    _count_students = 0
    
    
    def __init__(self, name, age = 18):
        self.name = name
        self.age = age
        Student._count_students+=1

    # Dunder or magic методы - методы с двумя _ в начале и в конце
    # Эти методы обладают магическими свойствами
    
    # __str__ - применяется к обьекту при вызове str от него
    def __str__(self):
        return self.name + ", " + str(self.age)

In [68]:
a = Student('Петер', 21)
str(a)

'Петер, 21'

## Инкапсуляция
Механизм сокрытия, позволяющий разграничивать доступ к различным компонентам программы. 

In [138]:
class Student:
    _count_students = 0
    
    
    def __init__(self, name, age = 18):
        self.name = name
        self.age = age
        Student._count_students+=1

    def __str__(self):
        return self.name + ", " + str(self.age)
    
    # public 
    def method_a(self):
        self.d = 'a'
        return 'a'
    
    # protected
    def _method_b(self):
        return 'b'
    
    # private
    def __method_c(self):
        return 'c'

In [142]:
a = Student('Петер', 21)
# Сработает без проблем
a.d

AttributeError: 'Student' object has no attribute 'd'

In [72]:
# Сработает без проблем, но большинство IDE подчеркнёт как ошибку
a._method_b()

'b'

In [79]:
# Не сработает
a.__method_c()

AttributeError: 'Student' object has no attribute '__method_c'

In [80]:
# Но всё равно можно вызвать через <object>._<class_name><method_name>()
a._Student__method_c()

'c'

## Наследование
Концепция объектно-ориентированного программирования, согласно которой абстрактный тип данных может наследовать данные и функциональность некоторого существующего типа, способствуя повторному использованию компонентов программного обеспечения.

class <дочерний класс>(<родительский класс>)

In [107]:
class A:
    def __init__(self):
        print('Привет, мир!')
        
class B(A):
    pass

b = B()

Привет, мир!


In [108]:
class C:
    
    # Члены можно переопределять
    def __init__(self):
        print("Привет, жестокий мир")

In [109]:
c = C()

Привет, жестокий мир


In [116]:
# issubclass(X, Y) возвращает True, если X - подкласса класса Y
issubclass(C, A), issubclass(B, A)

(bool, int, object)

## Полиморфизм
Свойство системы, позволяющее использовать объекты с одинаковым интерфейсом без информации о типе и внутренней структуре объекта. 

In [117]:
class Animal:
    def __init__(self):
        pass
    
    def bark(self):
        pass
    
class Cat(Animal):
    def bark(self):
        return "MEOW"
    

class Dog(Animal):
    def bark(self):
        return "BARK"
    

def do_voice(animal):
    print(animal.bark())
    
kraft = Dog()
do_voice(kraft)

mikki = Cat()
do_voice(mikki)

BARK
MEOW


## Декораторы
Это функции, оборачивающие другие функции

In [121]:
# import - подключение библеотек, как #include c/с++
import time 


def timeit(func):
    def wrapper_func(*args, **kwargs):
        start = time.time() # time.time текущее время в секундах отсчитывая c 1970 года
        func(*args, **kwargs)
        end = time.time()
        print(f'Elapsed time: {end - start}')
    return wrapper_func
    
    
def count(a = 10_000_000):
    for i in range(a):
        pass
    
timed_count = timeit(count)
timed_count(10_000_000)

Elapsed time: 0.1904282569885254


In [145]:
def f():
    return True, False
a, _ = f()
a

True